In [2]:
# Import necessary libraries for news extraction.
import requests
import json
import os
from dotenv import load_dotenv
from newspaper import Article

In [3]:
# Defining headers
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'
}

# URL of the original news article
article_url = 'https://www.aljazeera.com/news/2024/7/16/trump-receives-heros-welcome-in-first-appearance-since-assassination-bid'

In [4]:
# Creating the Session
session = requests.session()

# Fetching the news from the web
try:
    response = session.get(article_url, headers=headers, timeout=10)
    if response.status_code == 200:
        article = Article(article_url)
        article.download()
        article.parse()
        
        print(f'Title : {article.title}')
        print(f'Text : {article.text}')
        
    else:
        print(f'Failed to fetch article at {article_url}')
        
except Exception as e:
    print(f'Error occured while fetching article at {article_url}: {e}')

Title : Trump receives hero’s welcome at RNC in first appearance since being shot
Text : Donald Trump has received a hero’s welcome at the Republican National Convention, the former president’s first public appearance since he narrowly survived an attempted assassination.

Trump arrived at the Fiserv Forum in Milwaukee, Wisconsin to a rapturous reception on Monday, two days after he was struck in the ear by a would-be assassin’s bullet during a campaign rally in Butler, Pennsylvania.

Republican delegates stood and cheered as Trump, sporting a thick bandage over his right ear, entered the stadium to the sound of country music star Lee Greenwood singing “God Bless the USA”.

Trump, who appeared visibly moved, did not address the convention, but smiled and waved to the crowd as supporters chanted “Fight! Fight! Fight” and pumped their fists – a reference to the former president’s defiant reaction in the moments after he was shot.

After surveying the crowd, Trump took a seat in a VIP box

In [5]:
# Loading the environment variables
load_dotenv()

# Loading the API KEY for Huggingface from the environment variable
HF_API_KEY = os.environ['HF_API_KEY']

In [6]:
# Importing necessary libraries for News Summarizer
from langchain_huggingface.llms.huggingface_endpoint import HuggingFaceEndpoint
from langchain.prompts.prompt import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import field_validator, BaseModel, Field
from typing import List

In [7]:
# Defining the Pydantic class to be used as ouput parser
class ArticleSummary(BaseModel) : 
    title: str = Field(description="Title of the article")
    summary: List[str] = Field(description="Bulleted list summary of the article")
    
    # Field validator to check if the summary has more then three lines or not. The summary is expected to be of more than three lines.
    @field_validator('summary')
    def check_lines(cls, list_of_lines):
        if len(list_of_lines) < 3:
            raise ValueError("Generated summary has less than three bullet points.")
        return list_of_lines

In [8]:
# Creating the output parser object.
parser = PydanticOutputParser(pydantic_object=ArticleSummary)

In [9]:
# The prompt Template for LLM
propmt_template = PromptTemplate(
    input_variables=['article_title','article_text'],
    template='''
        You are an advanced AI assistant that summarizes online articles.

        Here's the article you want to summarize.

        ==================
        Title: {article_title}

        {article_text}
        ==================

        {format_instructions}
    ''',
    partial_variables={'format_instructions': parser.get_format_instructions()}
)

In [10]:
# The actual prompt to be passed to the LLM
prompt = propmt_template.format(article_title=article.title, article_text=article.text)

In [11]:
# Initializing the LLM via HuggingFace.
# The model used is Mixtral - 7B - Instruct. (A chat Model).
llm = HuggingFaceEndpoint(
    huggingfacehub_api_token=HF_API_KEY,
    repo_id='mistralai/Mixtral-8x7B-Instruct-v0.1',
    temperature=0.2
)

/home/ramakrishna/My Space/Data Science/Projects/News Article Summarizer/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/ramakrishna/.cache/huggingface/token
Login successful


In [12]:
# Extracting the response from the LLM
response = llm.invoke(prompt)
print(response)


    {
      "title": "Trump receives hero’s welcome at RNC in first appearance since being shot",
      "summary": [
        "Donald Trump received a hero's welcome at the Republican National Convention (RNC) in Milwaukee, Wisconsin.",
        "This was his first public appearance since he narrowly survived an attempted assassination at a campaign rally in Butler, Pennsylvania.",
        "Trump arrived to cheers and chants of 'Fight! Fight! Fight' and 'USA', and took a seat in a VIP box with some of his children and Ohio Senator JD Vance.",
        "Multiple speakers addressed the attack, with South Carolina Senator Tim Scott invoking religious imagery.",
        "Trump is due to formally accept the Republican nomination on Thursday, setting up a rerun of the 2020 contest against Democratic President Joe Biden.",
        "Trump has signalled that unity will be a key theme of the four-day event, but some speakers at the RNC delivered strident messages.",
        "Biden said on Monday t

In [13]:
# Parsing the response using the ouput parser.
parsed_response = parser.parse(response)
print(parsed_response)

title='Trump receives hero’s welcome at RNC in first appearance since being shot' summary=["Donald Trump received a hero's welcome at the Republican National Convention (RNC) in Milwaukee, Wisconsin.", 'This was his first public appearance since he narrowly survived an attempted assassination at a campaign rally in Butler, Pennsylvania.', "Trump arrived to cheers and chants of 'Fight! Fight! Fight' and 'USA', and took a seat in a VIP box with some of his children and Ohio Senator JD Vance.", 'Multiple speakers addressed the attack, with South Carolina Senator Tim Scott invoking religious imagery.', 'Trump is due to formally accept the Republican nomination on Thursday, setting up a rerun of the 2020 contest against Democratic President Joe Biden.', 'Trump has signalled that unity will be a key theme of the four-day event, but some speakers at the RNC delivered strident messages.', "Biden said on Monday that it had been a mistake to tell donors that Trump should be put 'in a bull’s-eye'

In [14]:
# Parsed Reponse (Title and the Bullet Points.)
print(parsed_response.title)
for item in parsed_response.summary:
    print('-',item)

Trump receives hero’s welcome at RNC in first appearance since being shot
- Donald Trump received a hero's welcome at the Republican National Convention (RNC) in Milwaukee, Wisconsin.
- This was his first public appearance since he narrowly survived an attempted assassination at a campaign rally in Butler, Pennsylvania.
- Trump arrived to cheers and chants of 'Fight! Fight! Fight' and 'USA', and took a seat in a VIP box with some of his children and Ohio Senator JD Vance.
- Multiple speakers addressed the attack, with South Carolina Senator Tim Scott invoking religious imagery.
- Trump is due to formally accept the Republican nomination on Thursday, setting up a rerun of the 2020 contest against Democratic President Joe Biden.
- Trump has signalled that unity will be a key theme of the four-day event, but some speakers at the RNC delivered strident messages.
- Biden said on Monday that it had been a mistake to tell donors that Trump should be put 'in a bull’s-eye', but defended his por